In [3]:
from flask import Flask
from flask_socketio import SocketIO, emit
from flask_cors import CORS
import time
import random

app2 = Flask(__name__)
socketio2 = SocketIO(app2, cors_allowed_origins="http://127.0.0.1:8080")
CORS(app2)

@socketio2.on('note-mix')
def note_mix(data):
    try:
        # 提取金额和货币类型
        currency = data['parameters']['currency']
        amount_data = data['parameters']['amount']
        amount_without_comma = amount_data.replace(',', '')
        amount = int(amount_without_comma)
        # 检查金额是否是100的倍数
        if amount % 100 != 0:
            emit('note-mix-response', {
                "event": "note-mix",
                "parameters": {
                    "success": False,
                    "actionCode": "re-enter-data",
                    "language": "en",
                    "vg": "Sorry, amount not in multiple of 100"
                }
            })
        else:
            # 如果金额合法，可以继续处理，返回成功的响应（根据你的需求调整）
            emit('note-mix-response', {
            "event": "note-mix",
            "parameters": {
                    "success": True,
                    "actionCode": "proceed",
                    "language": "en",
                    "vg": "Amount is valid"
                }
            })

    except Exception as e:
        # 错误处理
        emit('note-mix-response', {
            "event": "error",
            "message": str(e)
        })

@socketio2.on('cash-withdrawal')
def cash_withdrawal(data):
    try:
        currency = data['parameters']['currency']
        amount_data = data['parameters']['amount']
        amount_without_comma = amount_data.replace(',', '')
        amount = int(amount_without_comma)
        accountType = data['parameters']['accountType'].lower()
        receiptRequested = data['parameters']['receiptRequested']
        if accountType in ['saving account', 'checking account'] and amount and currency and receiptRequested is False:
            print("cash-withdrawal-response")
            # 处理提现请求
            emit('cash-withdrawal-response', {
                "event": "transaction-started"
            })
            time.sleep(random.randint(1, 5))
            emit('event', {'action': 'event', 'parameters': {'actionCode': '41', 'vg': 'please take money'}})

            time.sleep(random.randint(1, 5))
            emit('event', {'action': 'event', 'parameters': {'actionCode': '42', 'vg': 'please take card'}})
            
            time.sleep(random.randint(1, 5))
            emit('event', {'action': 'event', 'parameters': {'actionCode': '43', 'vg': 'end-transaction'}})
            

        elif accountType in ['saving account', 'checking account'] and amount and currency and receiptRequested is True:
            print("cash-withdrawal-response")
            
            state = 'transaction_started'
            
            while True:
                if state == 'transaction_started':
                    emit('cash-withdrawal-response', {"event": "transaction-started"})
                    #time.sleep(random.randint(1, 5))
                    state = 'please_take_money'  # 更新状态到下一个
                    
                elif state == 'please_take_money':
                    emit('event', {'action': 'event', 'parameters': {'actionCode': '41', 'vg': 'please take money'}})
                    
                    state = 'please_take_card'  # 更新状态到下一个
                elif state == 'please_take_card':
                    emit('event', {'action': 'event', 'parameters': {'actionCode': '42', 'vg': 'please take card'}})
                    
                    state = 'please_take_receipt'  # 更新状态到下一个
                elif state == 'please_take_receipt':
                    emit('event', {'action': 'event', 'parameters': {'actionCode': '43', 'vg': 'please take receipt'}})
                    
                    state = 'end_transaction'  # 更新状态到下一个
                elif state == 'end_transaction':
                    emit('end-transaction', {"action": "end-transaction"})
                    break  # 结束循环
            
        else:
            emit('cash-withdrawal-response', {
                "event": "transaction-failed",
                "message": "Invalid parameters"
            })
    except Exception as e:
        # 错误处理
        emit('cash-withdrawal-response', {
            "event": "error",
            "message": str(e)
        })

@socketio2.on('close-session')
def close_session(data):
    print(data['action'])


@socketio2.on('AI-Agent-ready')
def AI_Agent_ready(data):
    print(data['message'])
    emit('AI-Agent-ready-response', {
        "event": "AI-Agent-ready",
        "message": "AI-Agent is ready"
    })
    

if __name__ == '__main__':
    socketio2.run(app2, host='0.0.0.0', port=5002,allow_unsafe_werkzeug=True)  # 可以指定端口


AI Agent is ready
close-session
AI Agent is ready
